In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile, os, urllib.request, glob, math, shutil, sys
import torchvision
import torch.optim as optim
from torchvision import datasets, models, transforms
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, precision_recall_fscore_support, roc_auc_score, accuracy_score
import sklearn.metrics
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import PIL
import PIL.Image

sys.path.append('drive/MyDrive')
import modelArchs

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resume = 0

Mounted at /content/drive




Οι εικόνες είναι αποθηκευμένες στο drive σε zip.

Υπάρχουν 16 zip files, ένα για κάθε κλάση από το train και test dataset.

Κάνω extract τις εικόνες.


In [ ]:
os.makedirs('train_imgs')
os.makedirs('val_imgs')

prefix = 'drive/MyDrive/'

for i in ['train_imgs', 'val_imgs']:
  for j in ['MEL','NV','BCC','AK','BKL','DF','VASC','SCC']:

    name = i + '_' + j
    src_zip = prefix + name + '.zip'
    dest_zip = i + '/' + j

    with zipfile.ZipFile(src_zip, 'r') as zip_ref:
      zip_ref.extractall(dest_zip)

    print("Finished with: ", name)

Κάνω load το μοντέλο που έγινε train χωρίς τα μετα-δεδομένα.Στη συνέχεια ορίζω το τελικό νευρωνικό δίκτυο που θα γίνει train στις εικόνες + μετα-δεδομένα.

In [ ]:
PATH = 'drive/MyDrive/mobilenet_sa_derm.pt'
pretrained_model = modelArchs.get_MobileNetSa(8,False)
pretrained_model.load_state_dict(torch.load(PATH)['model'])

metaModel = modelArchs.get_metaModel_MobileNets(pretrained_model,11,8)
metaModel.to(device)

Load metadata.

In [ ]:
train_dataset = pd.read_csv('drive/MyDrive/train_dataset_meta_derm.csv',converters={'sex': pd.eval, 'anatom_site_general': pd.eval, 'age_approx': pd.eval})
test_dataset = pd.read_csv('drive/MyDrive/val_dataset_meta_derm.csv',converters={'sex': pd.eval, 'anatom_site_general': pd.eval, 'age_approx': pd.eval})
print(len(train_dataset))
print(len(test_dataset))
train_dataset.head()

Ορίζω διάφορα transformations για να κάνουμε augment το training set, ώστε να πετύχουμε καλύτερο generalization του μοντέλου.

In [ ]:
transforms_train = transforms.Compose([
    transforms.RandomResizedCrop(size = 224, scale = (0.4,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness = [0.7, 1.3], contrast = [0.7, 1.3], saturation = [0.7, 1.3], hue = [-0.1,0.1]),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transforms_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

Ορισμός υπερ-παραμέτρων.

In [ ]:
epochs = 30
batch_len = 48

criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(metaModel.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma=0.2)

if resume:
  checkpoint = torch.load('drive/MyDrive/mobilenet_sa_metadata_derm.pt')
  metaModel.load_state_dict(checkpoint['model'])
  optimizer.load_state_dict(checkpoint['optimizer'])
  scheduler.load_state_dict(checkpoint['scheduler'])
  epoch_count = checkpoint['epoch']
  print(scheduler.get_last_lr())
else:
  epoch_count = 1

print(epoch_count)

Υλοποιώ το δικό μου dataset, καθώς θα θέλαμε, εκτός από την εικόνα, να γίνονται load και τα αντίστοιχα metadata.

In [ ]:
features_list = ['age_approx', 'anatom_site_general', 'sex']

class ImageFolderWithFileNames(datasets.ImageFolder):

    def __init__(self, root_dir, train_transformations, current_dataset, features_list):

      super().__init__(root_dir,train_transformations)

      self.current_dataset = current_dataset
      self.features_list = features_list

    def __getitem__(self, index):

      path, target = self.imgs[index]

      filename = path[path.rfind('/') + 1:]
      metadata_features = self.get_metadata_features(filename)

      img = self.loader(path)
      if self.transform is not None:
        img = self.transform(img)
      if self.target_transform is not None:
        target = self.target_transform(target)

      return metadata_features,img,target

    def get_metadata_features(self,filename):

      features = []

      index_row = self.current_dataset.index[self.current_dataset['image'] == filename[:-4]].tolist()[0]#get index of the row that corresponds to the image

      feature_cols = self.current_dataset.iloc[index_row][self.features_list].to_numpy().flatten()

      for i in feature_cols:
        for j in list(i):
          features.append(j)

      return torch.tensor(features)

Ορίζουμε weights για κάθε κλάση, ώστε να αντιμετωπίσουμε το πρόβλημα του imbalanced dataset.

In [ ]:
def get_weights_for_imgs(images, n_categories):

    n_images = len(images)

    imgs_per_category = [0] * n_categories
    for _, image_category in images:
      imgs_per_category[image_category] += 1

    weight_per_category = [0] * n_categories # έχουμε 8 κατηγορίες
    for i in range(n_categories):
      weight_per_category[i] = 1 / float(imgs_per_category[i])#float(n_images) / float(imgs_per_category[i])

    weights = [0] * n_images
    for idx, (_, image_category) in enumerate(images):
        weights[idx] = weight_per_category[image_category]

    return weights

train_dataset_imgs = ImageFolderWithFileNames('train_imgs', transforms_train, train_dataset, features_list)
test_dataset_imgs = ImageFolderWithFileNames('val_imgs', transforms_test, test_dataset, features_list)

len_train_dataset_imgs = len(train_dataset_imgs)
len_test_dataset_imgs = len(test_dataset_imgs)

print("Length of train_dataset: ", len_train_dataset_imgs)
print("Length of test dataset: ", len_test_dataset_imgs)

weights = get_weights_for_imgs(train_dataset_imgs.imgs, len(train_dataset_imgs.classes))
weights = torch.DoubleTensor(weights)
train_sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))

train_dataloader_imgs = torch.utils.data.DataLoader(train_dataset_imgs, batch_size = batch_len, sampler = train_sampler, pin_memory=True)
test_dataloader_imgs = torch.utils.data.DataLoader(test_dataset_imgs, batch_size = 16, shuffle=False)

Ορίζω συναρτήσεις για υπολογισμό του accuracy και του confusion matrix πάνω στο test set. Στη συνέχεια θα δημιουργήσω και άλλες συναρτήσεις για υπολογισμό F1-score, sensitivity, specificity, precision.

In [ ]:
def test_model(model,test_dataloader):

  test_acc = 0.0
  pred_list = []
  label_list = []
  output_probs = []
  metrics = []

  for i, (metadata_features, imgs, labels) in enumerate(test_dataloader,1):

    label_list.extend(labels.data.tolist())

    metadata_features = metadata_features.to(device)
    imgs = imgs.to(device)
    labels = labels.to(device)

    outputs = model(metadata_features,imgs)

    probs = torch.softmax(outputs.data.cpu(), dim = 1).tolist()
    output_probs.extend(probs)

    pred =  torch.max(outputs, 1)[1].data.cpu().tolist()
    pred_list.extend(pred)

  conf_matrix = confusion_matrix(label_list , pred_list)

  metrics.append(accuracy_score(label_list, pred_list))
  metrics.extend(precision_recall_fscore_support(label_list,pred_list, average = 'macro'))
  metrics.append(roc_auc_score(label_list,output_probs, average = 'macro', multi_class = 'ovr'))
  metrics.extend(precision_recall_fscore_support(label_list,pred_list, average = 'weighted'))
  metrics.append(roc_auc_score(label_list,output_probs, average = 'weighted', multi_class = 'ovr'))

  return metrics, conf_matrix

Το βασικό train loop.

In [ ]:
for epoch in range(epoch_count, epochs + 1):

  metaModel.train()

  train_running_loss = 0

  #Training phase
  print("Starting epoch: " , epoch)

  for i, (metadata_features, imgs, labels) in enumerate(train_dataloader_imgs,1):

      metadata_features = metadata_features.to(device)
      imgs = imgs.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()
      outputs = metaModel(metadata_features,imgs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      train_running_loss += loss.detach().item()

  print('Epoch: %d | Loss: %.4f ' %(epoch, train_running_loss / i))
  scheduler.step()

  metaModel.eval()

  try:
    metrics,conf_matrix = test_model(metaModel,test_dataloader_imgs)
    print('Test metrics on epoch %d:' %(epoch))
    print(metrics)
    print("Confusion matrix: ")
    print(conf_matrix)
  except:
    print("Error in calculating metrics")

  try:
    checkpoint = {'epoch': epoch+1, 'model': metaModel.state_dict(), 'optimizer': optimizer.state_dict(), 'scheduler': scheduler.state_dict()}
    checkpoint_fn = 'drive/MyDrive/checkpoint' + str(epoch) + '.pt'
    torch.save(checkpoint, checkpoint_fn)
  except:
    print("Error in writing general checkpoint")
  if epoch - epoch_count >= 5:
    break